# Set-Up

In [180]:
import numpy as np
import codecs

In [181]:
hex_dict={'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'a':10,'b':11,'c':12,'d':13,'e':14,'f':15}
S=[0,2,10,9,6,4,14,13,1,7,15,8,11,12,3,5]
p=[0,5,10,15,4,9,14,3,8,13,2,7,12,1,6,11]

# Round Constants

def roundconst(i):
    r=[[[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,1]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,2]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,4]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,8]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,16]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,32]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,64]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,128]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,27]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,54]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,108]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,40]]]
    return r[i]

In [195]:
#Print State Funciton
def print_s(state,message):
  if debug==True:
    print(message)
    print()
    for i in range(4):
        for j in range(4):
          t=str(hex(state[i][j]))[2:]
          if len(t)<2:
            t='0'+t
          print(t[0],' ',t[1],' ',end='')
        print()
    print()
  else:
    return None

# Functions for encryption 

In [183]:
#S-box
S=[0,2,10,9,6,4,14,13,1,7,15,8,11,12,3,5]
def Sbox(n):
  a=str(hex(n))[2:]
  if len(a)<2:
          a='0'+a
  a1=hex_dict[a[0]]
  a2=hex_dict[a[1]]
  r=S[a1]*16+S[a2]
  return r


In [184]:
#L-box

def lbox(state):
    state_8=[[],[],[],[]]
    for i in range(4):
      for j in range(4):
        a=str(hex(state[i][j]))[2:]
        if len(a)<2:
          a='0'+a
        a1=hex_dict[a[0]]
        a2=hex_dict[a[1]]
        state_8[i].append(a1)
        state_8[i].append(a2)
    poly = [0, 0b1000, 0b0011, 0b1111, 0b0101, 0b1111, 0b0011, 0b1000]
    value=[]
    for c in range(4):
      for _ in range(8):
          x = state_8[c][0]
          for i in range(8):
              x ^= MultiplyGF16(state_8[c][i], poly[i])
          state_8[c].pop(0)
          state_8[c].append(x)
    for e in range(4):
      for f in range(4):
        state[e][f]=16*state_8[e][2*f]+state_8[e][2*f+1]
    return state

def MultiplyGF16(a, b, p=0b10011):
    result = 0
    for _ in range(4):
        result ^= (b & 1) * a
        a <<= 1
        a ^= (a >> 4) * p
        b >>= 1
    return result


In [185]:
#Shift Column

def ShiftColumns_128(state):
  lstate=[]
  for i in range(4):
    for j in range(4):
      lstate.append(state[i][j])
  plstate=[0]*16
  for i in range(16):
    plstate[p[i]]=lstate[i]
  value=[]
  for i in range(4):
    value.append(plstate[4*i:4*i+4])
  return value

In [196]:
def Mysterion128(key, m, NR):
  temp=''
  # adding key initially
  print_s(m,'Initially Message is:')
  print() 

  state = np.array( [x ^ y for x, y in zip(key, m)] )
  #print(state)
  if (debug or rounds):
    print('After Adding K_0:')
    print()
    for i in range(4):
      for j in range(4):
        t=str(hex(state[i][j]))[2:]
        if len(t)<2:
          t='0'+t
        print(t[0]+' '+t[1],' ',end='')
      print() 

  for round in range(1, NR + 1):
    if debug:
      print('------------Round No'+' : '+str(round)+'------------------')
      print()
    # S-box
    for i in range(4):
      for j in range(4):
        tp=Sbox(state[i][j])
        state[i][j]=tp
    print_s(state,'After S-box Operation:')

    # L-box
    for i in range(4):
      for j in range(4):
        state=lbox(state)
    print_s(state,'After L-box Operation:')

    # ShiftColumns 128-bit
    state = ShiftColumns_128(state)
    print_s(state,'After Shift Column:')

    #key addition
    state = np.array( [x ^ y for x, y in zip(key, state)])
    print_s(state,'After Key Addition:')

    #round Constant
    rc=roundconst(round-1)
    state = np.array( [x ^ c for x, c in zip(state, rc)])
    print_s(state,'After Round Constant:')

    if rounds==True:
      print()
      print('Round No : '+str(round))
      for i in range(4):
        for j in range(4):
          t=str(hex(state[i][j]))[2:]
          if len(t)<2:
            t='0'+t
          print(t[0]+' '+t[1],' ',end='')
        print()
      print()
  
  print('Ciphertext is: ')
  for i in range(4):
      for j in range(4):
        t=str(hex(state[i][j]))[2:]
        if len(t)<2:
          t='0'+t
        temp+=t
  temp=temp.upper()
  print(temp)
  return temp

# Functions For Decryption

In [187]:
def sbox_rev(n):
    a=str(hex(n))[2:]
    if len(a)<2:
            a='0'+a
    a1=hex_dict[a[0]]
    a2=hex_dict[a[1]]
    
    r=S.index(a1)*16+S.index(a2)
    return r

In [188]:
def rev_lbox(state):
    state_8=[[],[],[],[]]
    for i in range(4):
      for j in range(4):
        a=str(hex(state[i][j]))[2:]
        if len(a)<2:
          a='0'+a
        a1=hex_dict[a[0]]
        a2=hex_dict[a[1]]
        state_8[i].append(a1)
        state_8[i].append(a2)
    
    poly = [0b1000, 0b0011, 0b1111, 0b0101, 0b1111, 0b0011, 0b1000, 0]
    value=[]
    for c in range(4):
      for _ in range(8):
          x = state_8[c][7]
         
          for i in range(8):
              x ^= MultiplyGF16(state_8[c][i], poly[i])
              
          state_8[c].pop(7)
          state_8[c]=[x]+state_8[c][:]

    for e in range(4):
      for f in range(4):
        state[e][f]=16*state_8[e][2*f]+state_8[e][2*f+1]

    return state

In [189]:
def ShiftColumns_128_inv(state):
    lstate=[]
    for i in range(4):
      for j in range(4):
        lstate.append(state[i][j])
    plstate=[0]*16
    for i in range(16):
      plstate[i]=lstate[p[i]]
    value=[]
    for i in range(4):
      value.append(plstate[4*i:4*i+4])
    return value

In [197]:
def InvMysterion128(c, key, NR):
  state=c
  for round in range(NR, 0, -1):
    if rounds==True:
      print()
      print('Round No : '+str(round))
      for i in range(4):
        for j in range(4):
          t=str(hex(state[i][j]))[2:]
          if len(t)<2:
            t='0'+t
          print(t[0]+' '+t[1],' ',end='')
        print()
      print()

    if debug:
      print('----------Round No'+' : '+str(round)+'----------')
      print_s(state,'')


    rc=roundconst(round-1)

    for h in range(4):
      for g in range(4):
        f=state[h][g]
        state[h][g]=f^rc[h][g]
    print_s(state,'After Rev-Constant:')

    #revrse key
    state = np.array( [a^b for a, b in zip(state, key)])
    print_s(state,'After Rev-Key:')
    
    #revrse shift-row
    state = ShiftColumns_128_inv(state)
    print_s(state,'After Rev-ShiftRow:')

    # L-box inverse
    for i in range(4):
      for j in range(4):
        state=rev_lbox(state)
    print_s(state,'After Rev-L-box:')

    # S-box inverse
    for i in range(4):
      for j in range(4):
        tp=sbox_rev(state[i][j])
        state[i][j]=tp
    print_s(state,'after Rev-sbox')


  #intial key removal
  state = np.array( [x ^ y for x, y in zip(state, key)] )
  if (debug or rounds):
    print('After Removing K_0:')
    print()
    for i in range(4):
      for j in range(4):
        t=str(hex(state[i][j]))[2:]
        if len(t)<2:
          t='0'+t
        print(t[0]+' '+t[1],' ',end='')
      print() 
  
  print('Message is:')
  temp=''
  for i in range(4):
      for j in range(4):
        t=str(hex(state[i][j]))[2:]
        if len(t)<2:
          t='0'+t
        temp+=t
        print(t[0]+' '+t[1],' ',end='')
  temp=temp.upper()
  print(temp)
  return temp

# Encryption and Decryption

In [191]:
def encrypt(m, k, r):

    m = list(bytearray.fromhex(m))
    k = list(bytearray.fromhex(k))

    m = np.array([m[i*4:(i+1)*4] for i in range(4)])
    k = np.array([k[i*4:(i+1)*4] for i in range(4)])

    ct = Mysterion128(k, m, r)
    
    return ct

In [192]:
def decrypt(c, k, r):
    c = list(bytearray.fromhex(c))
    k = list(bytearray.fromhex(k))

    # spliting message in blocks 
    c = np.array([c[i*4:(i+1)*4] for i in range(4)])
    k = np.array([k[i*4:(i+1)*4] for i in range(4)])

    cpt = InvMysterion128(c, k, r)
    return cpt

# Running

In [200]:
inp='SHA 69 is best'

#String To Hex
message="".join(["{:02x}".format(ord(c)) for c in inp])
#Padding
if len(message)<32:
  message=message+'0'*(32-len(message))
if len(message)>32:
  message=message[:32]

message=message.upper()

key='0205060752F3E1F2132435465B6C7D88'
r=12
debug=False
rounds=True
ciphertext= encrypt(message,key,r)



After Adding K_0:

5 1  4 d  4 7  2 7  
6 4  c a  c 1  9 b  
6 0  0 4  5 7  2 3  
2 8  1 8  7 d  8 8  

Round No : 1
0 0  1 6  d 0  1 1  
a 9  1 2  f 3  f d  
1 2  8 2  1 2  e b  
0 5  f 1  a 3  f 6  


Round No : 2
0 b  4 1  8 4  3 a  
8 5  2 8  9 d  3 4  
c f  7 7  1 7  2 4  
0 5  7 1  b 7  f d  


Round No : 3
e d  e 5  1 a  0 4  
7 6  5 2  1 4  9 1  
e 9  b e  d d  5 0  
2 b  2 b  0 e  1 7  


Round No : 4
a 0  d 5  5 f  b 1  
d 2  9 b  3 0  a 7  
8 1  b 2  9 6  6 b  
5 5  6 a  9 5  4 0  


Round No : 5
6 f  8 7  9 f  5 7  
9 a  0 0  0 c  d 2  
a 2  3 3  e 0  6 d  
1 9  b 2  4 b  4 e  


Round No : 6
b 5  1 6  d 0  0 6  
c 4  f c  c 0  6 c  
4 8  a c  6 8  e 0  
6 c  d 3  4 8  3 2  


Round No : 7
d d  b 2  7 4  c c  
5 1  a d  2 f  5 2  
8 4  7 4  5 b  8 b  
0 0  4 6  b 8  a 0  


Round No : 8
a 8  c 9  d 2  2 b  
1 6  d 4  7 6  e 8  
e 7  1 7  f f  6 2  
5 1  f 8  1 d  3 1  


Round No : 9
1 8  e 0  4 3  7 c  
4 5  5 c  d 6  9 6  
8 c  6 3  c d  c 9  
f 0  e e  6 e  b 8  


Roun

In [201]:
d=decrypt(ciphertext,key,r)

d_a = codecs.decode(d, "hex")
print('Ascii is: ',str(d_a,'utf-8'))

if len(inp)<16:
  d=d[:len(d)-2*(16-len(inp))]
  d_a = codecs.decode(d, "hex")
  print()
  print('Ascii(without garbage) is: ')
  print(str(d_a,'utf-8'))


Round No : 12
d 6  b 7  2 3  2 a  
3 f  1 a  5 1  9 3  
d 9  c 1  4 7  3 3  
1 7  c 6  0 8  a 6  


Round No : 11
5 b  2 1  b 3  6 7  
3 a  c 7  f e  0 9  
d 2  a e  f 6  9 3  
c 7  5 7  d 2  b 2  


Round No : 10
d a  0 2  0 5  9 c  
3 e  6 8  8 f  7 e  
0 8  8 e  e 6  d 8  
3 4  2 0  0 7  3 9  


Round No : 9
1 8  e 0  4 3  7 c  
4 5  5 c  d 6  9 6  
8 c  6 3  c d  c 9  
f 0  e e  6 e  b 8  


Round No : 8
a 8  c 9  d 2  2 b  
1 6  d 4  7 6  e 8  
e 7  1 7  f f  6 2  
5 1  f 8  1 d  3 1  


Round No : 7
d d  b 2  7 4  c c  
5 1  a d  2 f  5 2  
8 4  7 4  5 b  8 b  
0 0  4 6  b 8  a 0  


Round No : 6
b 5  1 6  d 0  0 6  
c 4  f c  c 0  6 c  
4 8  a c  6 8  e 0  
6 c  d 3  4 8  3 2  


Round No : 5
6 f  8 7  9 f  5 7  
9 a  0 0  0 c  d 2  
a 2  3 3  e 0  6 d  
1 9  b 2  4 b  4 e  


Round No : 4
a 0  d 5  5 f  b 1  
d 2  9 b  3 0  a 7  
8 1  b 2  9 6  6 b  
5 5  6 a  9 5  4 0  


Round No : 3
e d  e 5  1 a  0 4  
7 6  5 2  1 4  9 1  
e 9  b e  d d  5 0  
2 b  2 b  0 e  1 7  


Round 